# Test raster dataset for adding to the dashboard
This is a helper notebook for adding datasets to the dashboard

The idea is that: a successful visualization of the data in the map in this notebook, would result in a successful visualization in the dashboard

## Instruction
- Add in the values of the variables in the cells that are specified as [Action Required]
- Run each of the cell in sequential order
- Check if the visualization looks correct in the map

In [47]:
# Installing required modules

!pip install folium requests

In [48]:
# Importing required libraries

from folium import Map, TileLayer
import requests
import urllib

In [49]:
# Some constants

TITILER_ENDPOINT = "https://baxpil3vd6.execute-api.us-east-1.amazonaws.com"
COG = "cog"
MOSAICJSON = "mosaicjson"
OPTIONS = [COG, MOSAICJSON]
TILE_URL_TEMPLATE = "{}/{}/tiles/{{z}}/{{x}}/{{y}}.png"
TILE_URL = { option: TILE_URL_TEMPLATE.format(TITILER_ENDPOINT, option) for option in OPTIONS  }

## <span style="color:red">[Action Required]</span>

Please fill in one of these two values, leave the other as is. If both is supplied, mosaicjson takes precedence

In [66]:
# User input

mosaic_json_url = "s3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/L8_tile_test.json"
# OR
tif_url = "s3://nasa-maap-data-store/file-staging/nasa-map/SRTMGL1_COD___001/N22W090.SRTMGL1.tif"

In [67]:
# Setting choice (mosaicjson/cog) based on the user input

url = mosaic_json_url if mosaic_json_url else tif_url

choice = None
if mosaic_json_url:
    choice = MOSAICJSON
elif tif_url:
    choice = COG
else:
    raise Exception("Please provide either a mosaic json url or a tif url")

## <span style="color:red">[Action Required]</span>

Add in the params that are needed, add in any extra params as required.

Consult [this link](https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/docs#/MosaicJSON/tile_mosaicjson_tiles__z___x___y__get) for the list of available params

In [68]:
# User input

# Add other params as required
params = {
    "url": url,
    "bidx": "6",
    "resampling_method": "nearest",
    "rescale": "0,1",
    "return_mask": "true",
    "colormap_name": "viridis"
}
params = {k: v for k, v in params.items() if v}

In [69]:
# Creating the tiles link using the user supplied params

query_string_params = urllib.parse.urlencode(params)

full_tiles_url = f"{TILE_URL[choice]}?{query_string_params}"

In [70]:
# Getting the bounds, min and max zoom levels based on the url provided

info = requests.get(
    f"{TITILER_ENDPOINT}/{choice}/info?url={url}"
).json()

# we need info['bounds'], info['maxzoom'], info['minzoom']
bounds = info.get('bounds')

In [71]:
# Initializing an OpenStreetMap with location set to the bounds of the input data

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2) if bounds else None,
)

In [72]:
# Creating a tile layer for the map

layer = TileLayer(
    tiles=full_tiles_url,
    attr="USGS",
    opacity=1,
    min_zoom=info.get("minzoom"),
    max_zoom=info.get("maxzoom")
)

In [73]:
# Adding the tile layer to the map
layer.add_to(m)

In [74]:
# Visualizing the map
m

In [75]:
full_tiles_url

'https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/mosaicjson/tiles/{z}/{x}/{y}.png?url=s3%3A%2F%2Fmaap-ops-dataset%2Fmaap-users%2Falexdevseed%2Flandsat8%2Fviz%2FL8_tile_test.json&bidx=6&resampling_method=nearest&rescale=0%2C1&return_mask=true&colormap_name=viridis'